# SnowEx SWE and Depth
### SWE from LayerData & SWE and Depth from PointData
In this Jupyter Notebook, we save SWE and depth data from the SnowEx database. From the LayerData class, we calculate SWE following the linked resource. We save the different dates when measurements are available. Next, from the PointData class, given the number of available measurements, we save every 1000th record.  

In [ ]:
from snowexsql.db import get_db # Import the function to get connect to the db
from snowexsql.conversions import query_to_geopandas # Import a useful function to format that data into a dataframe 
from datetime import date # Import some tools to build dates

from snowexsql.data import PointData # Import our class for the points table
from snowexsql.data import LayerData # Import our LayerData

import matplotlib.pyplot as plt

from geoalchemy2.shape import to_shape
import pandas as pd
import numpy as np

In [ ]:
# Connect to the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex' # This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

#### LayerData

A resource to calculate SWE: https://snowexsql.readthedocs.io/en/latest/gallery/plot_pit_swe_example.html

In [1]:
q = session.query(LayerData).filter(LayerData.type == 'density')
df = query_to_geopandas(q, engine)

# Convert density to float
df['value'] = df['value'].astype(float)

# Calculate SWE
swe_lambda = lambda row: row['value'] * (row['depth'] - row['bottom_depth']) / 100
df['swe'] = df.apply(swe_lambda, axis=1)

In [2]:
sites = df['site_id'].unique().tolist()

# Loop over data by site and date
swe_values = []
x_values = []
y_values = []
dates_list = []

for site in sites:
    ind1 = df['site_id'] == site
    dates = df['date'][ind1].unique().tolist()

    for date in dates:
        # Grab all density at this site and date
        ind2= df['date'] == date

        profile = df[ind1 & ind2]
        # Check if there is data on this date/site
        if len(profile.index) > 0:
            swe_values.append(profile['swe'].sum())
            x_values.append(profile['geom'].iloc[0].x)
            y_values.append(profile['geom'].iloc[0].y)
            dates_list.append(date)

In [3]:
df = pd.DataFrame(list(zip(swe_values, x_values, y_values, dates_list)),
               columns =['swe', 'x', 'y', 'date'])

In [4]:
df

,swe,x,y,date
0,568.500000,741960.000000,4.326644e+06,2020-01-31
1,535.900000,747070.000000,4.322511e+06,2020-02-04
2,541.266667,741866.000000,4.324050e+06,2020-02-11
3,408.766667,742054.000000,4.324454e+06,2020-02-08
4,349.100000,747792.000000,4.323957e+06,2020-01-28
...,...,...,...,...
493,124.933333,570820.786742,4.842961e+06,2020-02-12
494,155.466667,570823.155458,4.842965e+06,2020-02-20
495,275.186667,570819.105448,4.842967e+06,2020-02-26
496,354.666667,570825.618469,4.842961e+06,2020-03-04


#### PointData
We want to retrieve snow depth and swe (maybe this is GPR swe). Every 1000 points (subselect).

In [ ]:
# Pre-processing (just checking)
qry = "SELECT DISTINCT site_name FROM sites" # Form a typical SQL query and use python to populate the table name
results = engine.execute(qry) # Then we execute the sql command and collect the results
out = ', '.join((row['site_name'] for row in results)) # Create a nice readable string to print the site names using python 
print(out + '\n') # Create a nice readable string to print the site names using python 

# Get the unique datanames in the table
results = session.query(PointData.type).distinct().all()
print('Available types = {}'.format(', '.join([r[0] for r in results])))

# Get the unique instruments in the table
results = session.query(PointData.instrument).distinct().all()
print('Available types = {}'.format(', '.join([str(r[0]) for r in results])))

In [ ]:
%%time

full_values = []
full_dates = []
full_instruments = []
full_latitudes = []
full_longitudes = []
full_elevations = []

for i in range(len(np.unique(dates_list))):
    # Pick a dataset
    dataset = 'depth' #depth

    # Site name
    site_name = "Grand Mesa"

    # Pick a date
    collection_date = date(dates_list[i].year, dates_list[i].month, dates_list[i].day)

    # Get a session
    engine, session = get_db(db_name)

    qry = session.query(PointData)
    qry = qry.filter(PointData.type == dataset)
    qry = qry.filter(PointData.date == collection_date)

    # Execute the query and convert to geopandas in one handy function
    df = query_to_geopandas(qry, engine)
    # print(list(df.columns))
    len_df = np.arange(len(df))
    df2 = df[(len_df % 1000 == 1)]
    
    full_values.append(list(df2['value']))
    full_dates.append(list(df2['date']))
    full_instruments.append(list(df2['instrument']))
    full_latitudes.append(list(df2['latitude']))
    full_longitudes.append(list(df2['longitude']))
    full_elevations.append(list(df2['elevation']))
    
    # how many did we retrieve?
    print(i, f'{len(df2.index)} records')
session.close()

0 170 records
1 605 records
2 3 records
3 58 records
4 608 records
5 86 records
6 605 records
7 91 records
8 2 records
9 605 records
10 2 records
11 608 records
12 3 records
13 170 records
14 91 records
15 108 records
16 3 records
17 3 records


In [14]:
# we flatten our list of lists
flat_full_values = [x for xs in full_values for x in xs]; print(len(flat_full_values))
flat_full_dates = [x for xs in full_dates for x in xs]; print(len(flat_full_dates))
flat_full_instruments = [x for xs in full_instruments for x in xs]; print(len(flat_full_instruments))
flat_full_latitudes = [x for xs in full_latitudes for x in xs]; print(len(flat_full_latitudes))
flat_full_longitudes = [x for xs in full_longitudes for x in xs]; print(len(flat_full_longitudes))
flat_full_elevations = [x for xs in full_elevations for x in xs]; print(len(flat_full_elevations))

16146
16146
16146
16146
16146
16146


In [15]:
#change depth to SWE if saving SWE
df_PointData = pd.DataFrame(list(zip(flat_full_values, flat_full_dates, flat_full_instruments, 
                                     flat_full_latitudes, flat_full_longitudes, flat_full_elevations)),
               columns =['depth', 'date', 'instrument', 'latitude', 'longitude', 'elevation']) 

In [16]:
df_PointData.head()

,SWE,date,instrument,latitude,longitude,elevation
0,229.0,2020-01-31,Mala 800 MHz GPR,39.024950,-108.169552,3106.40
1,240.0,2020-01-31,Mala 800 MHz GPR,39.024962,-108.169678,3103.62
2,275.0,2020-01-31,Mala 800 MHz GPR,39.025108,-108.169702,3101.36
3,326.0,2020-01-31,Mala 800 MHz GPR,39.024778,-108.169581,3105.89
4,291.0,2020-01-31,Mala 800 MHz GPR,39.025262,-108.169362,3106.17


In [17]:
# saving our dataframe so that we do not need to load it everytime
# df_PointData.to_pickle("PointData_SWE.pkl")
df_PointData.to_pickle("PointData_depth.pkl")

#### Matching LayerData and PointData

In [18]:
PointData_SWE = pd.read_pickle("PointData_SWE.pkl")
print(PointData_SWE)

         SWE        date         instrument   latitude   longitude  elevation
0      229.0  2020-01-31   Mala 800 MHz GPR  39.024950 -108.169552    3106.40
1      240.0  2020-01-31   Mala 800 MHz GPR  39.024962 -108.169678    3103.62
2      275.0  2020-01-31   Mala 800 MHz GPR  39.025108 -108.169702    3101.36
3      326.0  2020-01-31   Mala 800 MHz GPR  39.024778 -108.169581    3105.89
4      291.0  2020-01-31   Mala 800 MHz GPR  39.025262 -108.169362    3106.17
...      ...         ...                ...        ...         ...        ...
16141  203.0  2020-02-04  Mala 1600 MHz GPR  39.031349 -108.139465    3147.60
16142  216.0  2020-02-04  Mala 1600 MHz GPR  39.030843 -108.139429    3143.02
16143  161.0  2020-02-04  Mala 1600 MHz GPR  39.031217 -108.139108    3141.90
16144  237.0  2020-02-04  Mala 1600 MHz GPR  39.031177 -108.139100    3141.01
16145  273.0  2020-02-04  Mala 1600 MHz GPR  39.031174 -108.138983    3141.70

[16146 rows x 6 columns]


In [ ]:
PointData_depth = pd.read_pickle("PointData_depth.pkl")
print(PointData_depth)